## Logistic Regression – Using More Intuitive Feature Selection for Easier Interpretation

In this step, I’m transitioning from the EDA phase into building a logistic regression model.  
Instead of using the full one-hot encoded dataset with 50+ features, I’m intentionally **selecting a smaller, more interpretable set of features** that are both relevant and easier to explain in terms of their relationship to the target variable (`y`).

**Rationale for this approach:**
- **Interpretability:** A smaller set of features makes it easier to understand how each variable influences the prediction.
- **Simplicity:** Reduces complexity and multicollinearity issues that can occur with many dummy variables.
- **Focus on meaningful predictors:** Using variables that have clear business or contextual meaning.

**Features selected:**
- `age` – Age of the individual.
- `previous` – Number of previous contacts with the client.
- `emp.var.rate` – Employment variation rate.
- `cons.price.idx` – Consumer price index.
- `cons.conf.idx` – Consumer confidence index.
- `euribor3m` – 3-month Euribor interest rate.
- `nr.employed` – Number of employees.
- `prior_contact` – Binary flag if the client had been previously contacted.
- `campaign_capped` – Number of contacts made during the current campaign (capped).

**Target variable:**
- `y` – Binary outcome indicating if the client subscribed to the term deposit (1) or not (0).

**Next steps:**
1. Split data into training and testing sets.
2. Scale numeric features for logistic regression.
3. Train the model.
4. Evaluate performance (accuracy, classification report, confusion matrix).
5. Optionally, interpret coefficients to understand feature importance.


In [21]:
#lets first import pandas and read from the cleaned dataset
import pandas as pd


df = pd.read_csv('cleaned_bank.csv')

1.Feature selection (My reduced intuitive feature list).

2.Train/test split.

3.Scaling.

4.Model fitting.

5.Evaluation.

1.) Feature Selection

In [22]:
#Feature selection for logistic regression
#We will use the following features for our logistic regression model:
#1. age, previous, emp.var.rate, cons.price.idx, cons.conf.idx, euribor3m, nr.employed

features_to_keep = [
    'age',
    'previous',
    'emp.var.rate',
    'cons.price.idx',
    'cons.conf.idx',
    'euribor3m',
    'nr.employed',
    'prior_contact',
    'campaign_capped'
]




### Feature Selection for Logistic Regression

In this step, we define the set of features that will be used as inputs for our logistic regression model.  
The selected features are chosen because they capture key aspects of the client’s profile, economic indicators,  
and campaign history that may influence the outcome.

**Selected Features:**
1. **age** – Age of the client.  
2. **previous** – Number of previous contacts with the client.  
3. **emp.var.rate** – Employment variation rate (economic indicator).  
4. **cons.price.idx** – Consumer price index.  
5. **cons.conf.idx** – Consumer confidence index.  
6. **euribor3m** – Euribor 3-month rate (financial indicator).  
7. **nr.employed** – Number of employees (macro-level economic indicator).  
8. **prior_contact** – Indicates whether the client was contacted previously in the campaign.  
9. **campaign_capped** – A capped version of the campaign variable to limit outliers in contact attempts.  


In [23]:
#keep only selected features compare against the target variable 'y'
X = df[features_to_keep].copy () # Features for the model
y = df['y'].astype(int)  # Convert target variable to integer (0 or 1)

#type(X), type(y)
# ensure binary columns are ints (0/1) not bool/object
X[['prior_contact', 'campaign_capped']] = X[['prior_contact', 'campaign_capped']].astype('category')

### Preparing Features and Target Variable for Logistic Regression

After selecting the features, the next step is to separate the input variables (**X**) and the target variable (**y**) to train our logistic regression model.

**Steps:**
1. **Select Features (X):**  
   - `X = df[features_to_keep].copy()` creates a new dataframe containing only the chosen features.  
   - This prevents accidental changes to the original dataframe while modeling.  

2. **Define Target (y):**  
   - `y = df['y'].astype(int)` extracts the target column (`y`) and converts it to integers (0 or 1).  
   - This ensures the target is properly formatted for binary classification.  

3. **Ensure Correct Data Types for Binary Features:**  
   - `X[['prior_contact', 'campaign_capped']] = ... .astype('category')`  
   - Converts these features into categorical variables (instead of boolean or object).  
   - This helps the model and preprocessing pipelines treat them correctly.  


Identify Column Types
Identify which columns are numeric and which are categorical

In [24]:
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

print("Numeric columns:", numeric_cols)
print("Categorical columns:", categorical_cols)
print('target variable:', y.name + ' 0 or 1:', y.dtype)


Numeric columns: ['age', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
Categorical columns: ['prior_contact', 'campaign_capped']
target variable: y 0 or 1: int32


2.) Train/test split.

In [25]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
# test size 20% of rows go to test set, 80% to train set
# random_state ensures reproducibility of the split
# stratify ensures that the proportion of classes in y is maintained in both train and test sets


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42, stratify=y)

#
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(31462, 9) (7866, 9) (31462,) (7866,)


**Train/Test Split — Output Summary**
-------------------------------------

**X_train**: feature matrix used to train the model (rows = training examples, cols = features).  
**y_train**: target labels for those training examples.  
**X_test**: feature matrix held out for evaluating the trained model.  
**y_test**: true labels for the test rows used only to measure performance.  

---

### 🔑 Key differences and why they matter
- **Purpose:**  
  - `X_train` / `y_train` → learn model parameters.  
  - `X_test` / `y_test` → assess how the learned model generalizes to unseen data.  

---

### 📊 What the Output Means
- `X_train` → 31,462 rows × 9 features (80% of the data)  
- `X_test`  →  7,866 rows × 9 features (20% of the data)  
- `y_train` → 31,462 labels (matches X_train rows)  
- `y_test`  →  7,866 labels (matches X_test rows)  

---

### 🧮 Why the Numbers Make Sense
- **Total rows:**  
  31,462 + 7,866 = 39,328 total rows in dataset.  

- **80/20 split check:**  
  - Train: 39,328 × 0.8 ≈ 31,462  ✅  
  - Test:  39,328 × 0.2 ≈  7,866  ✅  

- **Same features in train/test:**  
  Both X_train and X_test have exactly 9 columns (features).  


Now I need to scale continous features, fit logistic regression on X_train/y_train

3.) Scaling

In [27]:
#Now lets do some preprocessing of the data
#Preprocessing is important for machine learning models to ensure that the data is in the right format
#Preprocessing = cleaning, imputing, encoding, scaling, etc.
#Typical steps, 1.) train test split, 2.) categorical encoding, 3.) scaling, 4.) model training

#We already did the train test split above, now we will do categorical encoding and scaling
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

#time to do one-hot encoding for categorical features
preprocessor = ColumnTransformer (

    transformers = [
        ('num', StandardScaler(), numeric_cols),  # Scale numeric columns
        
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols) # One-hot encode categorical columns, ignore unknown categories during transformation
    ]
    , remainder='passthrough'  # Keep numeric columns as they are
)

#Now we do a pipeline to combine preprocessing with the model
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[
    ('preprocessor',preprocessor),  # Preprocessing step
    ('clf', LogisticRegression(class_weight = 'balanced',max_iter=1000))  # Logistic Regression model, balannced class weights to handle class imbalance

])

4.) Fit Model

5.) Evaluate